# Create a Tamil Tokenizer using Google SentencePiece

Initial version: Ravi Annaswamy (April 6, 2019)

### What is tokenization?

A 'word' in a language is usually a composite word. An example in English would be the word: 'unconditionally'.

How many parts does it have?

One could split it as: un-condition-al-ly

We could say four:
- prefix: un
- root is: condition
- first suffix: al
- second suffix is: ly


### Why Tokenize?

Why should we split? In order to make sense of its meaning. The prefixes and suffixes modify the core meaning to put it to some specific use. Some prefixes are syntactic, some semantic. 

This is even more composite in Indic languages where EVERY verb and noun is an inflection form. In English, verbs take on tense and sometimes person. In almost all Indian languages, verbs take on tense, person, count, gender.
In Tamil additionally, nouns take on count, proposition too and verb form takes on the conjunction.

Take an example tamil word: அமைந்துள்ளதால்
This means: "because it is situated" 
 அமைந்துள்ளதால் = அமை ந்த ுள்ள து ஆல்
 அமை (situate) ந்த (d) ுள்ள (is) து (it) ஆல் (because)
 
 To give another concrete illustration, a word such as, 'வந்திருந்தானா?' really is a phrase (Had he come?) splits to 'வ' 'ந்த' 'இரு' 'ந்த' 'ஆன்' 'ஆ', compactly encoding almost one syllable each for the verb, past tense, participle, past tense of the participle, gender and person and number, query marker respectively!

Seeing a word as parts is so essential for understanding. These parts are called as tokens and splitting a word to its pieces is called tokenization.

A translation engine between Tamil and English can now correlate word piece meanings and learn the root is verb!

### How to create a tokenizer?

In traditional NLP, tokenizers and lemmatizers were initially hand constructed. Then they were machine learned from large corpus of data that has been tagged by humans. With modern machine learning one can construct reasonable tokenizers just using information theory and optimization principles. 

Google sentencepiece software (similar to BPE) learns a vocabulary that optimally can construct any word in a corpus.

From raw unsegmented text, if you give it a vocabulary size, say 20000 words, it figures out 20000 word-parts that will give optimal coverage of entire text.

 That is 20000 words are given token ids, rest of the words are composed using this.


 Sentencepiece (and similar technique called Byte Pair Encoding (BPE) truly serve well as an automatically constructed lemmatizer and tokenizer.

##  Step 1. Install Google Sentencepiece software. This automatically figures out word-parts (roots, suffixes, prefixes).

In [ ]:
!pip install sentencepiece

## Step 2. Get list of files from Tamil Wikipedia (or any corpus you would like) to serve as input for sentencepiece

### If you want to test this quickly, you can point to tawiki_small instead of large, to get a smaller dataset to train the sentence piece model.

In [13]:
TA_WIKI_PATH = '../corpus/tawiki/tawiki_large/'

In [15]:
import glob
flist = glob.glob(TA_WIKI_PATH+'*/*')

In [16]:
import random

fname=random.choice(flist)
print(fname)
with open(fname, encoding='utf-8') as f:
    text = f.read()
print(text[:1000])

../01_corpus/tawiki/tawiki_large/AA/AA_wiki_65.txt
<doc id="45501" url="https://ta.wikipedia.org/wiki?curid=45501" title="அணுக்கரு விசை">
அணுக்கரு விசை

அணுக்கரு விசை ("Nuclear force") எனப்படுவது இரண்டு அல்லது அதற்கு மேற்பட்ட அணுக்கருனிகளுக்கிடையே ஏற்படும் விசை ஆகும். அணுவின் உட்கருவில் உள்ள நேர் மின்னூட்டம் கொண்ட புரோட்டான்களுக்கிடையில் உள்ள மின்னூட்ட விலக்கல் விசையை விட அதிகமான அளவில் இந்த அணுக்கரு விசை செயற்பட்டு புரோட்டான்களையும் மின்னூட்டம் அற்ற நியூட்ரான்களையும் பிணைக்க வைக்கின்றது.

1932 ஆம் ஆண்டில் ஜேம்ஸ் சாட்விக் என்பவரால் நியூட்டாரன் கண்டுபிடிக்கப்பட்டதில் இருந்து அணுக்கரு விசை அணுக்கருவியலில் முக்கியத்துவம் பெற்றது.

1935 ஆம் ஆண்டில் ஹிடெக்கி யுக்காவா என்பவர் அணுக்கரு விசை பற்றி முதன் முதலாக எடுத்துக் கூறினார். அவரது கூற்றுப்படி, இரு அணுக்கருனிகளுக்கிடையே தொடர்ந்தாற் போல் போசோன்கள் (மீசோன்கள்) என்ற துகள் பரிமாற்றம் நடைபெறுகிறது. எனினும், மீசோன் தத்துவம் தற்போது இவ்விசையின் அடிப்படைக் கொள்கை என ஏற்றுக் கொள்ளப்படாவிட்டாலும், இந்த மீசோன் துகள் பரிமாற்றக் கொள்கை "நியூட்ரான்-நியூ

## Step 3. Make this filelist into a long comma-separated string to feed sentencepiece trainer.

In [17]:
flist2=','.join(flist)
flist2[:200]

'../01_corpus/tawiki/tawiki_large/AE/AE_wiki_46.txt,../01_corpus/tawiki/tawiki_large/AE/AE_wiki_34.txt,../01_corpus/tawiki/tawiki_large/AE/AE_wiki_04.txt,../01_corpus/tawiki/tawiki_large/AE/AE_wiki_42.'

## Step 4. Call Sentencepiece Train function, asking it to create a spm model with 8000 word parts.

8000 may seem very small, but I found this optimal for language modeling, translation software. 
I suspect that this is because almost all tamil verbs are two syllables, and tense, number, gender, person are all 
encoded in one syllable each!

In [18]:
import sentencepiece as spm

In [19]:
spm.SentencePieceTrainer.Train(f'--input={flist2} --model_prefix=tamil_spm_8k --vocab_size=8000')

True

## Step 5. Now Load this model into a processor object and ask it to tokenize pieces of text string.

In [20]:
spt = spm.SentencePieceProcessor()

In [21]:
spt.Load("tamil_spm_8k.model")

True

## Now you can pass any text and it will return tokens!

In [22]:
spt.EncodeAsPieces('அமைந்துள்ளதால்')

['▁அமைந்துள்ள', 'தால்']

In [23]:
tok_text = spt.EncodeAsPieces(text[:1250])
print(' '.join(tok_text))

▁< doc ▁id =" 45 50 1 " ▁ url =" https :// ta . wikipedia . org / wiki ? curi d = 45 50 1 " ▁title =" அ ணு க்கரு ▁விசை "> ▁அணுக்கரு ▁விசை ▁அணுக்கரு ▁விசை ▁(" N uc le ar ▁for ce ") ▁எனப்படுவது ▁இரண்டு ▁அல்லது ▁அதற்கு ▁மேற்பட்ட ▁அணுக்கரு னி களுக்கிடையே ▁ஏற்படும் ▁விசை ▁ஆகும் . ▁அணு வின் ▁உட் கரு வில் ▁உள்ள ▁நேர் ▁மின்ன ூட்ட ம் ▁கொண்ட ▁புரோ ட்டான் களுக்க ிட ையில் ▁உள்ள ▁மின்ன ூட்ட ▁விலக்க ல் ▁விசை யை ▁விட ▁அதிகமான ▁அளவில் ▁இந்த ▁அணுக்கரு ▁விசை ▁செயற் பட்டு ▁புரோ ட்டான் களையும் ▁மின்ன ூட்ட ம் ▁அ ற்ற ▁நியூ ட்ரா ன் களையும் ▁பிணை க்க ▁வைக்க ின்றது . ▁19 32 ▁ஆம் ▁ஆண்டில் ▁ஜேம்ஸ் ▁சா ட் வி க் ▁என்பவரால் ▁நியூ ட்டா ரன் ▁கண்டுபிடிக்க ப்பட்ட தில் ▁இருந்து ▁அணுக்கரு ▁விசை ▁அணுக்கரு வியலில் ▁முக்கியத்துவம் ▁பெற்றது . ▁1935 ▁ஆம் ▁ஆண்டில் ▁ஹி டெ க்கி ▁யு க்கா வா ▁என்பவர் ▁அணுக்கரு ▁விசை ▁பற்றி ▁முதன் ▁முதலாக ▁எடுத்துக் ▁கூறினார் . ▁அவரது ▁கூற்றுப்படி , ▁இரு ▁அணுக்கரு னி களுக்கிடையே ▁தொடர்ந்த ா ற் ▁போல் ▁போ சோ ன் கள் ▁( மீ சோ ன் கள் ) ▁என்ற ▁துகள் ▁பரிமாற்ற ம் ▁நடைபெறுகிறது . ▁எனினும் , ▁மீ சோ ன் ▁தத்த

## Please how it has split words - you get the picture!
'▁நியூ ட்ரா ன் களையும்',
'▁பாதுகாப்ப',
 'வர்',
 'களை'
'▁அணி',
 'ந்த',
 'வர்',
 'களாகவும்',
 '▁இருப்ப',
 'வர்',
 'களாகவும்',

In [24]:
!head -50 tamil_spm_8k.vocab

<unk>	0
<s>	0
</s>	0
.	-3.04693
,	-3.73844
▁	-4.33228
"	-4.65909
ம்	-4.69299
="	-4.75747
க்	-4.94869
ப்	-4.99399
doc	-5.07301
கள்	-5.07628
்	-5.3333
)	-5.38481
▁(	-5.42036
த்	-5.42682
▁ஒரு	-5.49748
▁மற்றும்	-5.50963
d	-5.58061
/	-5.64493
>	-5.69503
ன்	-5.70746
ta	-5.72214
ு	-5.73666
?	-5.73758
▁<	-5.73955
://	-5.75524
org	-5.75924
https	-5.76287
▁</	-5.76389
wikipedia	-5.76452
wiki	-5.76453
url	-5.7652
curi	-5.766
▁title	-5.76649
▁id	-5.76664
">	-5.77756
ல்	-5.78961
ும்	-5.8505
ர்	-5.85397
து	-5.85483
▁"	-5.89759
▁இந்த	-5.93694
ய	-5.97812
யில்	-6.02099
ச்	-6.02583
ஸ்	-6.03458
த்தில்	-6.07806
=	-6.10535


You just need the following two files (model and vocab) to use the learned tokenizer

In [25]:
!ls

tamil_spm_8k.model  Tokenizer for Tamil using Google sentencepiece.ipynb
tamil_spm_8k.vocab


## Step 6. Let us also train a 16000 word tamil word segmentation model for experiments.

In [26]:
spm.SentencePieceTrainer.Train(f'--input={flist2} --model_prefix=tamil_spm_16k --vocab_size=16000')

True

In [30]:
spt2 = spm.SentencePieceProcessor()
spt2.Load("tamil_spm_16k.model")

True

In [31]:
spt2.EncodeAsPieces('அமைந்துள்ளதால்')

['▁அமைந்துள்ள', 'தால்']

In [32]:
tok_text_16k = spt2.EncodeAsPieces(text[:1250])
print(' '.join(tok_text_16k))

▁< doc ▁id =" 45 501" ▁ url =" https :// ta . wikipedia . org / wiki ? curi d =45 501" ▁title =" அ ணு க்கரு ▁விசை "> ▁அணுக்கரு ▁விசை ▁அணுக்கரு ▁விசை ▁(" N uc le ar ▁for ce ") ▁எனப்படுவது ▁இரண்டு ▁அல்லது ▁அதற்கு ▁மேற்பட்ட ▁அணுக்கரு னி களுக்கிடையே ▁ஏற்படும் ▁விசை ▁ஆகும் . ▁அணு வின் ▁உட்கரு வில் ▁உள்ள ▁நேர் ▁மின்னூட்ட ம் ▁கொண்ட ▁புரோட்டான் களுக்கிடையில் ▁உள்ள ▁மின்னூட்ட ▁விலக்க ல் ▁விசை யை ▁விட ▁அதிகமான ▁அளவில் ▁இந்த ▁அணுக்கரு ▁விசை ▁செயற்பட்டு ▁புரோட்டான் களையும் ▁மின்னூட்ட ம் ▁அற்ற ▁நியூ ட்ரான் களையும் ▁பிணை க்க ▁வைக்க ின்றது . ▁1932 ▁ஆம் ▁ஆண்டில் ▁ஜேம்ஸ் ▁சா ட் விக் ▁என்பவரால் ▁நியூ ட்டா ரன் ▁கண்டுபிடிக்க ப்பட்ட தில் ▁இருந்து ▁அணுக்கரு ▁விசை ▁அணுக்கரு வியலில் ▁முக்கியத்துவம் ▁பெற்றது . ▁1935 ▁ஆம் ▁ஆண்டில் ▁ஹி டெ க்கி ▁யு க்கா வா ▁என்பவர் ▁அணுக்கரு ▁விசை ▁பற்றி ▁முதன் ▁முதலாக ▁எடுத்துக் ▁கூறினார் . ▁அவரது ▁கூற்றுப்படி , ▁இரு ▁அணுக்கரு னி களுக்கிடையே ▁தொடர்ந்த ாற் ▁போல் ▁போ சோ ன் கள் ▁( மீ சோ ன் கள் ) ▁என்ற ▁துகள் ▁பரிமாற்றம் ▁நடைபெறுகிறது . ▁எனினும் , ▁மீ சோ ன் ▁தத்துவம் ▁தற்போது ▁இவ்வி

## Step 7. Visually compare tokenization performance of 16k model and 8k model to see which seems natural for Tamil use.

In [33]:
# create normal test
test_text = '▁வித்யா ராஜா க்கள் ▁பெரும்பாலும் ▁உ க்கிர ▁மூர்த்தி களைப் ▁போல் ▁பல ▁முக ங்கள் , ▁கர ங்கள் ▁முதலியவற்ற ோடு ▁சித்திர க்க ப்படுகின்றனர் . ▁கையில் ▁ஆயுதங்கள் ▁ஏ ந்திய வாறு ம் ▁சில ▁சி ம ய ங்களில் ▁க பால ம் ▁மற்றும் ▁மிரு க ▁தோல் களை ▁அணிந்த வர்களாகவும் ▁தீ ப்பி ழ ம்பு கள் ▁சூழ ▁இருப்பவர்கள ாகவும் ▁காட்சியளிக்க ின்றனர் .'
test_text=test_text.replace(' ','').replace('▁',' ').strip()
print(test_text)


வித்யாராஜாக்கள் பெரும்பாலும் உக்கிர மூர்த்திகளைப் போல் பல முகங்கள், கரங்கள் முதலியவற்றோடு சித்திரக்கப்படுகின்றனர். கையில் ஆயுதங்கள் ஏந்தியவாறும் சில சிமயங்களில் கபாலம் மற்றும் மிருக தோல்களை அணிந்தவர்களாகவும் தீப்பிழம்புகள் சூழ இருப்பவர்களாகவும் காட்சியளிக்கின்றனர்.


In [35]:
tok_text_8k = spt.EncodeAsPieces(test_text)
print('8k tokenization:',' '.join(tok_text_8k))
print()
tok_text_16k = spt2.EncodeAsPieces(test_text[:1250])
print('16k tokenization:',' '.join(tok_text_16k))

8k tokenization: ▁வித்யா ராஜா க்கள் ▁பெரும்பாலும் ▁உ க்கிர ▁ மூர்த்தி களைப் ▁போல் ▁பல ▁முக ங்கள் , ▁கர ங்கள் ▁முதலிய வற்ற ோடு ▁சித்திர க்க ப்படுகின்றனர் . ▁கை யில் ▁ஆயுத ங்கள் ▁ஏ ந்திய வாறு ம் ▁சில ▁சி மய ங்களில் ▁க பால ம் ▁மற்றும் ▁மிருக ▁தோல் களை ▁அணி ந்த வர் களாகவும் ▁தீ ப்பி ழ ம்பு கள் ▁சூழ ▁இருப்ப வர் களாகவும் ▁காட்சி யளிக்க ின்றனர் .

16k tokenization: ▁வித்யா ராஜா க்கள் ▁பெரும்பாலும் ▁உக்கிர ▁மூர்த்தி களைப் ▁போல் ▁பல ▁முக ங்கள் , ▁கர ங்கள் ▁முதலியவற்ற ோடு ▁சித்திர க்க ப்படுகின்றனர் . ▁கையில் ▁ஆயுதங்கள் ▁ஏ ந்திய வாறு ம் ▁சில ▁சி மய ங்களில் ▁க பால ம் ▁மற்றும் ▁மிருக ▁தோல் களை ▁அணிந்த வர்களாகவும் ▁தீ ப்பி ழ ம்பு கள் ▁சூழ ▁இருப்பவர் களாகவும் ▁காட்சி யளிக்க ின்றனர் .


### What do you notice? Do you notice the differences?

Here is what I see: .
    ▁ஆயுத ங்கள் -> ▁ஆயுதங்கள்  (weapons)
    ▁அணி ந்த வர் களாகவும் -> ▁அணிந்த வர்களாகவும் [(they) as adorning (animal skin)].
    
    
The 8k resource-limited model correctly splits 'weapons' into weapon+s, 

the resource-rich 16K model allocation a new word for the plural form.

Same is the case for second word: 8k splits it to 4 parts 
 - core verb, 
 - past tense marker, 
 - third person marker, and 
 - number marker+proposition.
 
 The 16k model splits into 2 parts.
 I like the 8k better because it has teased out the latent variables better.
 
 



## Some themes in history and philosophy (without the politics)

Most Indian languages have solid grammatical foundations in morphology and phonetics.

While we normally think of grammars as specifying word classes (such as nouns, verbs and their modifiers adjectives, adverbs and degree markers etc) and sentence making rules, Indian grammars specify rules for word making and even sound making so that a language can be precisely spoken across vast land masses if needed without dialect distortions.

So the forms of Tamil and Sanskrit and their children are in some sense *designed* languages (or at the least *explicitly refined* languages).

Tholkappiyam (தொல்காப்பியம் http://www.tholkappiyam.org/) and Nannool (நன்னூல்) are ancient Tamil grammars explicitly specify how various modifiers adorn verbs or nouns in addition to specifying almost every aspect of word making, phrase making, sentence making and meaning making and language evolution. 

Panini's Astadhyayi (http://ashtadhyayi.com/sutraani/) likewise is an extremely concise grammar for Sanskrit and was one of the inspiration behind modern programming language design via Sassuire, BNF, Chomsky path. 

Panini takes grammar description to almost information theoretic obsession of a codebook, a minimal-code codebook - to the degree the text of his spec is not meant to be 'human' readable but like a book of mathematical formula, only describes latent entities and interactions that turn in to language later, through progressive refinement. In fact he moved beyond writing a 'descriptive' grammar to a 'generative grammar' which led to the CFG like spec. Sentence formulae that govern creation of ANY sentence.

As a result of such rich design in language and the ensuing regularity of the governed languages, even simple information theoretic analysis of raw text, could quickly reveals the core grammar of such languages. Latin, German and hundreds of other languages also have such properties and my guess is that using a small 'core' wordpart parameter will make their language models simpler too.

It is a matter of pride for all of us humans, that Indians, Chinese and Middle Eastern world had made such breakthroughs in systematization of written and spoken languages over 5000 years ago and had well specified grammars 2000 years ago. And also a matter of pride that Europeans and western world have built upon those breakthroughs to tie them to computation and programming language and now we are able to make one (Computation) help understand the other (Language). Final matter of joy that internet, globalization, computing, storage breakthroughs in the last two decades have established such an amount of knowledge sharing and collaborative tools that now every one across the world can help each other as members of one family working towards common goal of improving every one's life.

Later we will build language models for one language from each of major groups: Indian, European, Chinese, Russian, African families and try to do our own quick study of common roots and language geneologies. Now you know the value of human-created and curated treasure that is Wikipedia.


# Our Hope 

Past: Human grammarians created language models by studying language usage and theorizing on that. These grammars were then used to teach people how to communicate with each other quickly and without confusion.

Present: In the last 100 years, these ideas combined with computation and information theory, led to language models for communicating efficiently and effectively to a computer.

Present-present: We are at a point where machines are able to create language models from natural language using statistical techniques, with less and less human guidance, tedium and bias.

Future: We hope that computer generated grammars will be far more effective in parsing/understanding natural text because they can gain experience on vast vast amounts of data with less choice of bias and prejudice over long times. They could help humans create new era of multi-lingual and cross lingual unified grammars and our ability to communicate to each other across globe will get a huge lift, soon. 


## Back to topic, we have 2 tokenizers to choose from.

In [36]:
!ls

tamil_spm_16k.model  tamil_spm_8k.vocab
tamil_spm_16k.vocab  Tokenizer for Tamil using Google sentencepiece.ipynb
tamil_spm_8k.model


## Next step we will build a language model (which is a collection of automatically learning syntactic, semantic and pragmatic rules that help predict what word will come next in a sequence of a real-world sentence).

We will learn this using a powerful library called fastai from a generous genius, *Jeremy Howard* (https://en.wikipedia.org/wiki/Jeremy_Howard_(entrepreneur)) 

We will learn it from large wikipedia article collection which has wide coverage of articles, topics, sentence formats to inform our model.

# Done